In [ ]:
# Install (versions compatibles + évite le package `keras` standalone)
%pip install -U pip setuptools wheel
%pip install -U tabulate scikit-learn pandas numpy matplotlib seaborn gensim "tensorflow==2.16.1"

https://arxiv.org/abs/2007.12673 - Genetic Algorithm: Reviews, Implementations, and Applications - Tanweer Alam, Shamimul Qamar, Amit Dixit, Mohamed Benaida

# imports

In [1]:
# Suppression des avertissements liés à Scikit-learn
import warnings  # Masquer les avertissements (ex. : FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import gc  # garbage collector (32Gb suffit pas)

# Librairies générales
import pandas as pd  # Librairie pour la manipulation de données
import numpy as np  # Librairie pour le calcul numérique
import sys  # Fonctions et variables liées à l'interpréteur Python
import copy  # Création de copies d'objets
from numpy import mean, std  # Fonctions de calcul de moyenne et d'écart type
import zipfile  # Traitement de fichiers zip
import os  # Manipulation de fichiers et chemins

# Librairie affichage
import matplotlib.pyplot as plt  # Outils de visualisation 2D
from matplotlib import pyplot  # Interface de la bibliothèque Matplotlib
import seaborn as sns  # Bibliothèque de visualisation de données basée sur Matplotlib

# Scikit-learn pour l'évaluation des modèles
from sklearn.metrics import confusion_matrix  # Matrice de confusion
from sklearn.model_selection import KFold  # Outils de validation croisée
from sklearn.metrics import accuracy_score  # Calcul de l'accuracy
from sklearn.model_selection import train_test_split  # Découpage train/test

# TensorFlow et Keras
import tensorflow as tf  # Librairie de deep learning
import keras  # API haut niveau pour construire et entraîner des modèles de deep learning
from keras import layers  # Modules de couches pour construire des modèles Keras
from keras import models  # Outils pour créer des modèles Keras
from keras import optimizers  # Outils d'optimisation
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Générateur d'images pour l'augmentation des données
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping  # Rappels pour le suivi et l'arrêt précoce
from keras.layers import Input, Dense, Dropout, Flatten, Lambda  # Types de couches Keras
from keras.layers import Conv2D, MaxPooling2D  # Couches convolutionnelles et de pooling
from keras.preprocessing import image  # Outils de prétraitement d'images
from tensorflow.keras.models import Model, load_model  # Définition / chargement de modèles
from keras.datasets import fashion_mnist  # Jeu de données Fashion MNIST
from tensorflow.keras.utils import to_categorical  # Conversion en encodage one-hot
from tensorflow.keras.optimizers import SGD  # Optimiseur Stochastic Gradient Descent
from tensorflow.keras.applications.resnet50 import ResNet50  # Modèle ResNet50 pré-entraîné
from tensorflow.keras.preprocessing import image  # Prétraitement d'images pour les modèles Keras

def tf_cleanup(close_plots: bool = False):
    """Nettoyage agressif TF/GC entre folds/individus pour limiter l'accumulation."""
    if close_plots:
        try:
            plt.close('all')
        except Exception:
            pass
    try:
        tf.keras.backend.clear_session()
    except Exception:
        pass
    gc.collect()

# DataSet

## File declare

In [2]:
# Définition du répertoire cible
data_dir = "./data/dataset/sheep_cat_elephant_with_caption_600"
data_dir_img = os.path.join(data_dir, "images")
img_height, img_width = 224, 224
batch_size = 64

## Download

In [ ]:
# Création du répertoire s'il n'existe pas
os.makedirs(data_dir, exist_ok=True)

zip_file = "Data_sheep_cat_elephant_with_caption_600.zip"

#!wget https://www.lirmm.fr/~poncelet/Ressources/cnn_models.zip
!Powershell.exe -Command ((new-object System.Net.WebClient).DownloadFile('https://www.lirmm.fr/~poncelet/Ressources/Data_sheep_cat_elephant_with_caption_600.zip','Data_sheep_cat_elephant_with_caption_600.zip'))

# Extraction du fichier ZIP
with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(data_dir)

# Suppression du fichier ZIP après extraction pour économiser de l'espace
os.remove(zip_file)

# GA class

Modèle avec paramètres d'archi modifiables

In [3]:
class ModelInstance:
    def __init__(self, 
                 cLayers, # nombre de couches convolutionnelles
                 cDims, # [(nb_filtres : int, taille_filtres : (int, int), taille_pooling : (int, int))]
                 dLayers, # nombre de couches denses
                 dDims, # [nb_neurones : int]
                 dropout, # taux de dropout (ignoré si 0f)
                 input_shape, # (int, int, int)
                 output_shape, # int
                 name
                 ):
        # couche d'entrée
        input = Input(shape=input_shape, name="input")
        x = input
        # couches conv
        for i in range(cLayers):
            x = Conv2D(cDims[i][0], cDims[i][1], activation="relu", name=f"conv_{i+1}")(x)
            x = MaxPooling2D(cDims[i][2], name=f"pool_{i+1}")(x)
        # flatten
        x = Flatten(name="flatten")(x)
        # couches denses
        for i in range(dLayers):
            x = Dense((int(dDims[i])), activation="relu", name=f"dense_{i+1}")(x)
        # couche de sortie
        if dropout[0] > 0:
            x = Dropout(dropout[0], name="dropout")(x)
        output = Dense(output_shape, activation="softmax", name="output")(x)
        model = Model(inputs=input, outputs=output, name=name)
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        self.model = model
        self.cLayers = cLayers
        self.cDims = cDims
        self.dLayers = dLayers
        self.dDims = dDims
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.name = name
    def summary(self):
        return self.model.summary()
    def fit(self, x, y=None, **kwargs):
        return self.model.fit(x, y, **kwargs)
    def evaluate(self, x, y=None, **kwargs):
        return self.model.evaluate(x, y, **kwargs)

    def predict(self, x, **kwargs):
        return self.model.predict(x, **kwargs)

    def save(self, path):
        self.model.save(path)

    def load(self, path):
        self.model = load_model(path)

Gene + Modèle

In [4]:
class Entity:
    def __init__(self,
                 cDims,  # [(nb_filtres : int, taille_filtres : (int, int), taille_pooling : (int, int))]
                 dDims,  # [nb_neurones : int]
                 dropout,  # taux de dropout (ignoré si 0f)
                 input_shape,  # (int, int, int)
                 output_shape,  # int,
                 name
                 ):
        self.cLayers = len(cDims)
        self.cDims = cDims
        self.dLayers = len(dDims)
        self.dDims = dDims
        self.dropout = dropout
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.name = name

        # IMPORTANT: ne pas garder de modèle TF en mémoire entre générations.
        # On ne conserve que les gènes + métriques; le modèle est (re)construit à la demande.
        self.model_instance = None

        self.loss = None
        self.accuracy = None
        self.complexity = None

        # Complexité calculée 1 fois, sans conserver le modèle.
        self._ensure_model()
        self.complexity = float(self.model_instance.model.count_params())
        self.drop_model()

    def _ensure_model(self):
        if self.model_instance is None:
            self.model_instance = ModelInstance(
                self.cLayers, self.cDims, self.dLayers, self.dDims, self.dropout, self.input_shape, self.output_shape, self.name
            )
        return self.model_instance

    def drop_model(self, close_plots: bool = False):
        """Libère explicitement les ressources TF/Keras associées au modèle de l'entité."""
        try:
            if self.model_instance is not None:
                # casse les refs au graph/weights
                try:
                    self.model_instance.model.stop_training = True
                except Exception:
                    pass
                self.model_instance.model = None
        except Exception:
            pass
        self.model_instance = None
        tf_cleanup(close_plots=close_plots)
        return self

    def evaluate_fitness(self, test_data):
        self._ensure_model()
        self.loss, self.accuracy = self.model_instance.evaluate(test_data)
        return self.accuracy

    def compute_complexity(self):
        # trainable params sans l'optimizer; calculé quand un modèle existe
        self._ensure_model()
        self.complexity = float(self.model_instance.model.count_params())
        return self.complexity

    def summary(self):
        self._ensure_model()
        return self.model_instance.summary()

    def fit(self, x, y=None, **kwargs):
        self._ensure_model()
        return self.model_instance.fit(x, y, **kwargs)

    def evaluate(self, x, y=None, **kwargs):
        self._ensure_model()
        self.loss, self.accuracy = self.model_instance.evaluate(x, y, **kwargs)
        return self.loss, self.accuracy

    def predict(self, x, **kwargs):
        self._ensure_model()
        return self.model_instance.predict(x, **kwargs)

    def reset(self):
        # reset = reconstruire un modèle neuf (utile en k-fold)
        self.drop_model()
        self.model_instance = ModelInstance(
            self.cLayers, self.cDims, self.dLayers, self.dDims, self.dropout, self.input_shape, self.output_shape, self.name
        )
        self.loss = None
        self.accuracy = None
        return self

## Data + train

In [5]:
def load_dataset():
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir_img,
        validation_split=0.3,
        subset="training",
        seed=124,
        image_size=(img_height, img_width),
        batch_size=batch_size,
        label_mode="int",
        shuffle=True,
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir_img,
        validation_split=0.3,
        subset="validation",
        seed=124,
        image_size=(img_height, img_width),
        batch_size=batch_size,
        label_mode="int",
        shuffle=True,
    )

    X_train_list, y_train_list = [], []
    for x, y in train_ds:
        X_train_list.append(x.numpy())
        y_train_list.append(y.numpy())

    X_test_list, y_test_list = [], []
    for x, y in val_ds:
        X_test_list.append(x.numpy())
        y_test_list.append(y.numpy())

    X_train = np.concatenate(X_train_list, axis=0)
    y_train = np.concatenate(y_train_list, axis=0)
    X_test = np.concatenate(X_test_list, axis=0)
    y_test = np.concatenate(y_test_list, axis=0)

    # One-hot
    numClass = len(train_ds.class_names)
    y_train = to_categorical(y_train, num_classes=numClass)
    y_test = to_categorical(y_test, num_classes=numClass)

    return X_train, y_train, X_test, y_test

In [6]:
def clean_data(train, test):
    """
    Prétraitement des données : conversion en float, normalisation entre 0 et 1.

    Paramètres :
    - train : tableau de données d'entraînement
    - test : tableau de données de test

    Retourne :
    - train_norm : données d'entraînement normalisées
    - test_norm : données de test normalisées
    """
    # Conversion des entiers en floats pour permettre la normalisation
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')

    # Normalisation des valeurs entre 0 et 1
    train_norm /= 255.0
    test_norm /= 255.0

    return train_norm, test_norm

In [7]:
def evaluate_model(model, dataX, dataY, folds=5, epochs=10, keep_histories=False):
    """
    Évalue le modèle avec une validation croisée K-fold.

    Notes mémoire:
    - Un `Model` TF doit être détruit entre folds pour éviter l'accumulation de graph/optimizers.
    - On évite de conserver `History` en GA (keep_histories=False).
    """
    scores, losses = [], []
    histories = []
    kfold = KFold(n_splits=folds, shuffle=True, random_state=1)
    print(model.summary())

    for train_ix, test_ix in kfold.split(dataX):
        X_train, y_train = dataX[train_ix], dataY[train_ix]
        X_test, y_test = dataX[test_ix], dataY[test_ix]

        # Modèle neuf pour ce fold
        model = model.reset()
        callbacks = [EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=0)]

        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=callbacks
        )

        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        scores.append(float(acc))
        losses.append(float(loss))
        if keep_histories:
            histories.append(history)

        # Cleanup agressif après chaque fold
        try:
            del history
        except Exception:
            pass
        model.drop_model()

    # Affichage des statistiques de précision : moyenne et écart-type
    print(f'Précision : moyenne={np.mean(scores) * 100:.3f}% écart-type={std(scores) * 100:.3f}%, k={len(scores)}')
    model.accuracy = float(np.mean(scores)) if len(scores) else None
    model.loss = float(np.mean(losses)) if len(losses) else None
    # complexity est stable (calculée à l'init), pas besoin de rebuild ici
    return scores, histories if keep_histories else None

In [8]:
def plot_curves(histories):
    """
    Fonction pour afficher les courbes de loss et d'accuracy
    moyennees et ecart-types a travers les k-folds.

    Parametres :
    - histories (list) : Historique d'entrainement des differents plis K-folds.
    """
    if not histories:
        return

    # Aligne les historiques sur la longueur minimale (early stopping).
    min_len = min(len(h.history["loss"]) for h in histories)
    trimmed = []
    for h in histories:
        trimmed.append({k: v[:min_len] for k, v in h.history.items()})

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    epochs = range(min_len)

    mean_loss = np.mean([h["loss"] for h in trimmed], axis=0)
    std_loss = np.std([h["loss"] for h in trimmed], axis=0)
    mean_val_loss = np.mean([h["val_loss"] for h in trimmed], axis=0)
    std_val_loss = np.std([h["val_loss"] for h in trimmed], axis=0)

    mean_accuracy = np.mean([h["accuracy"] for h in trimmed], axis=0)
    std_accuracy = np.std([h["accuracy"] for h in trimmed], axis=0)
    mean_val_accuracy = np.mean([h["val_accuracy"] for h in trimmed], axis=0)
    std_val_accuracy = np.std([h["val_accuracy"] for h in trimmed], axis=0)

    train_color = 'blue'
    val_color = 'orange'

    ax1.plot(epochs, mean_loss, color=train_color, label='Train')
    ax1.fill_between(epochs, mean_loss - std_loss, mean_loss + std_loss, color=train_color, alpha=0.2)
    ax1.plot(epochs, mean_val_loss, color=val_color, label='Validation')
    ax1.fill_between(epochs, mean_val_loss - std_val_loss, mean_val_loss + std_val_loss, color=val_color, alpha=0.2)

    ax2.plot(epochs, mean_accuracy, color=train_color, label='Train')
    ax2.fill_between(epochs, mean_accuracy - std_accuracy, mean_accuracy + std_accuracy, color=train_color, alpha=0.2)
    ax2.plot(epochs, mean_val_accuracy, color=val_color, label='Validation')
    ax2.fill_between(epochs, mean_val_accuracy - std_val_accuracy, mean_val_accuracy + std_val_accuracy, color=val_color, alpha=0.2)

    ax1.set_title(f'Loss (k={len(histories)})')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    ax2.set_title(f'Accuracy (k={len(histories)})')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    plt.show()
    plt.close(fig)

In [9]:
def run_evaluation(folds, epochs, model, X_train, y_train, X_test, y_test, plot=True):
    """Évaluation interactive (debug) : peut afficher les courbes. Pour le GA, éviter plot=True."""
    print(model.summary())
    scores, histories = evaluate_model(model, X_train, y_train, folds, epochs, keep_histories=plot)
    if plot and histories is not None:
        plot_curves(histories)
    print(f'Précision : moyenne={mean(scores) * 100:.3f}% écart-type={std(scores) * 100:.3f}%, k={len(scores)}')
    # cleanup final (histoires/figures)
    tf_cleanup(close_plots=plot)

# GA RUN

## params

In [10]:
# Base Gene
input_shape = (img_height, img_width, 3)
cDims = [
    (6, (3, 3), (2, 2)),
    (32, (5, 5), (2, 2)),
    (6, (3, 3), (2, 2))
    ]
dDims = [20]
dropout = [0.0]
output_shape = 3
epochs = 20
folds = 10
# choices :
# 1. Add conv layer
# 2. pop conv layer
# 3. change a conv layer param
#   3.1 change nb_filtres
#   3.2 change taille_filtres
#   3.3 change taille_pooling
# 4. add dense layer
# 5. pop dense layer
# 6. change a dense layer param
#  6.1 change nb_neurones

# Pop params
num_pop = 40
num_gen = 10

In [11]:
def mutate(conv, cDims, dense, dDims, dropout):
    taille_filtres = [(3, 3), (5, 5), (7, 7)]
    nb_filtres = [6, 8, 16, 32, 64, 128]
    taille_pooling = [(2, 2), (3, 3)]
    neur_dense = [10, 20, 50, 100, 150, 200]
    dropout_values = [0.0, 0.5, 0.75]
    d = dense
    c = conv
    mutation_type = np.random.randint(1, 8)
    if mutation_type == 1: # add conv layer
        if c < 6:
            c += 1
            cDims.append((nb_filtres[0], taille_filtres[0], taille_pooling[0]))
        else:
            return mutate(c, cDims, d, dDims, dropout)
    elif mutation_type == 2: # pop conv layer
        if c > 2:
            c -= 1
            cDims.pop()
        else:
            return mutate(c, cDims, d, dDims, dropout)
    elif mutation_type == 3: # change a conv layer param
        layer_idx = np.random.randint(0, c)
        param_idx = np.random.randint(0, 3)
        if param_idx == 0: # change nb_filtres
            cDims[layer_idx] = (np.random.choice(nb_filtres), cDims[layer_idx][1], cDims[layer_idx][2])
        elif param_idx == 1: # change taille_filtres
            cDims[layer_idx] = (cDims[layer_idx][0], taille_filtres[np.random.randint(0, len(taille_filtres))], cDims[layer_idx][2])
        else: # change taille_pooling
            cDims[layer_idx] = (cDims[layer_idx][0], cDims[layer_idx][1], taille_pooling[np.random.randint(0, len(taille_pooling))])
    elif mutation_type == 4: # add dense layer
        if d < 2:
            d += 1
            dDims.append(neur_dense[0])
        else:
            return mutate(c, cDims, d, dDims, dropout)
    elif mutation_type == 5: # pop dense
        if d > 1:
            d -= 1
            dDims.pop()
    elif mutation_type == 6: # change dropout
        dropout = [np.random.choice(dropout_values)]
    else: # change a dense param
        if d > 0:
            layer_idx = np.random.randint(0, d)
            dDims[layer_idx] = np.random.choice(neur_dense)
        else:
            return mutate(c, cDims, d, dDims, dropout)
    return c, cDims, d, dDims, dropout


def mutate_unique(cDims, dDims, existing_configs, dropout):
    for _ in range(10):  # Limite le nombre de tentatives pour trouver une mutation unique
        new_config = mutate(len(cDims), cDims, len(dDims), dDims, dropout)
        valid = True
        for config in existing_configs:
            conv_match = False
            cDims_match = True
            if new_config[0] == config[0]: # conv
                conv_match = True
                for i in range(len(new_config[1])):
                    if new_config[1][i] != config[1][i]: # cDims
                        cDims_match = False
            dense_match = False
            dDims_match = True
            if new_config[2] == config[2]: # dense
                dense_match = True
                for i in range(len(new_config[3])):
                    if new_config[3][i] != config[3][i]: # dDims
                        dDims_match = False
            dropout_match = False
            if new_config[4] == config[4]: # dropout
                 dropout_match = True
            if conv_match and cDims_match and dense_match and dDims_match and dropout_match:
                valid = False
                break
        if valid:
            existing_configs.append(new_config)
            return new_config[1], new_config[3], new_config[4]
        else:
            return mutate_unique(cDims, dDims, existing_configs, dropout)

## Run

In [ ]:
# Chargement du jeu de données d'entraînement et de test
X_train, y_train, X_test, y_test = load_dataset()

# Prétraitement des données : nettoyage et normalisation
X_train, X_test = clean_data(X_train, X_test)

pop = [Entity(cDims, dDims, dropout.copy(), input_shape, output_shape, "0_0")]  # base pour controle
conf = [[len(cDims), cDims.copy(), len(dDims), dDims.copy(), dropout.copy()]]
for i in range(num_pop - 1):
    pop.append(Entity(*mutate_unique(cDims.copy(), dDims.copy(), conf, dropout), input_shape, output_shape, "0_"+str(i+1)))

for i in range(num_gen):
    # ENTRAINEMENT + EVAL

    print(f"Generation {i+1}")
    prev_best = pop[0]
    baseline_accuracy = prev_best.accuracy
    for entity in pop:
        # Un seul entrainement pour le meilleur precedent
        if entity is prev_best and baseline_accuracy is not None:
            continue
        # En GA: pas de courbes, pas d'histories -> beaucoup moins de RAM
        evaluate_model(entity, X_train, y_train, folds=folds, epochs=epochs, keep_histories=False)
        # Cleanup par individu
        entity.drop_model()

    # Clean RAM (génération)
    tf_cleanup(close_plots=True)

    # SELECTION (A AMELIORER)
    if baseline_accuracy is None:
        baseline_accuracy = pop[0].accuracy
    pop = [entity for entity in pop if entity.accuracy > baseline_accuracy]
    if (len(pop) == 0): # si rien est strictement meilleur, on garde prev_best 
        pop = [prev_best]
    # on trie par simplicite
    pop.sort(key=lambda x: x.complexity)
    # on choisi le plus simple
    best_entities = [pop[0]]

    print("Best entity :")
    print(f"Accuracy : {best_entities[0].accuracy:.3f}, Complexity : {best_entities[0].complexity:.3f}")
    print(best_entities[0].summary())
    print(f"param best: conv={best_entities[0].cLayers}, cDims={best_entities[0].cDims}, dense={best_entities[0].dLayers}, dDims={best_entities[0].dDims}, dropout={best_entities[0].dropout}")
    # Important: ne pas garder le modèle en RAM après l'affichage
    best_entities[0].drop_model()

    # MUTATRON !!!!

    # generer new pop en mutant la best
    pop = [best_entities[0]]
    conf = [[best_entities[0].cLayers, best_entities[0].cDims.copy(), best_entities[0].dLayers, best_entities[0].dDims.copy(), best_entities[0].dropout.copy()]]
    for j in range(num_pop - 1):
        pop.append(Entity(
            *mutate_unique(
                best_entities[0].cDims.copy(),
                best_entities[0].dDims.copy(),
                conf,
                best_entities[0].dropout.copy()
            ),
            input_shape,
            output_shape,
            str(i+1)+"_"+str(j+1)
        ))

print("Final best entity :")
print(f"Accuracy : {best_entities[0].accuracy:.3f}, Complexity : {best_entities[0].complexity:.3f}")
print(best_entities[0].summary())
print(f"param best: conv={best_entities[0].cLayers}, cDims={best_entities[0].cDims}, dense={best_entities[0].dLayers}, dDims={best_entities[0].dDims}, dropout={best_entities[0].dropout}")
best_entities[0].drop_model()
tf_cleanup(close_plots=True)

Found 1800 files belonging to 3 classes.
Using 1260 files for training.
Found 1800 files belonging to 3 classes.
Using 540 files for validation.

Generation 1


Model: "0_0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1 (Conv2D)                 │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 107, 107, 32)   │         4,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling2D)           │ (None, 53, 53, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 51, 51, 6)      │         1,734 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_3 (MaxPooling2D)           │ (None, 25, 25, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3750)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │        75,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 81,817 (319.60 KB)

 Trainable params: 81,817 (319.60 KB)

...

Model: "1_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1 (Conv2D)                 │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 74, 74, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 70, 70, 32)     │         4,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling2D)           │ (None, 35, 35, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 33, 33, 6)      │         1,734 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_3 (MaxPooling2D)           │ (None, 16, 16, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │       153,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,737 (627.88 KB)

 Trainable params: 160,737 (627.88 KB)

 Non-trainable params: 0 (0.00 B)